<a href="https://colab.research.google.com/github/simonefmr/teste/blob/main/ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www3.bcb.gov.br/ranking/historico.do

# **Pós Graduação Ciência de Dados - PUC Rio**
# **MVP Sprint I: Análise de Dados e Boas Práticas**
## Aluna: Simone de Fátima Marques Ramos

## **1. Introdução**

### **1.1. Problema de aprendizado supervisionado ou não supervisionado e Hipóteses sobre o problema**

### **1.2. Definição do Problema** 

## **2. Importação de pacotes e bibliotecas**

## **3. Carregamento dos dados**

### **3.1. Condições ou restrições impostas para selecionar os dados**

### **3.2. Definição dos atributos**

## **4. Análise de Dados**

### **4.1. Limpeza preliminar dos dados**